# Imports

In [1]:
import os, re, math
from dataclasses import dataclass
from typing import List, Tuple, Dict, Optional
import numpy as np
import pandas as pd
from tqdm import tqdm
import pdfplumber
from transformers import pipeline, AutoTokenizer
import unicodedata

C:\Users\semde\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] Kan opgegeven procedure niet vinden'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


# Config

In [2]:
# === Config: RobBERT v2 + neutral band en gewichten (titel/lead/body) ===
ROB_BERT_SENTIMENT = "DTAI-KULeuven/robbert-v2-dutch-sentiment"

DATA_DIR = "data"
OUT_CSV  = "sentiment_results.csv"

from dataclasses import dataclass

@dataclass
class Weights:
    title: float = 1.0
    lead:  float = 1.0
    body:  float = 1.0   # body weer meewegen

WEIGHTS = Weights()

# Neutral band voor beslissing op basis van signed score (p_pos - p_neg)
NEUTRAL_BAND = 0.1

# Chunk-instellingen (gebruik je al elders)
MAX_TOKENS = 400
STRIDE     = 50

# Helpers

In [3]:
LEXIS_HEADER_PATTERNS = [
    r"^About LexisNexis.*",
    r"^Privacy Policy.*",
    r"^Terms .* Conditions.*",
    r"^Copyright.*",
    r"^User Name:.*",
    r"^Date and Time:.*",
    r"^Job Number:.*",
    r"^Documents \\(\\d+\\).*",
    r"^Client/Matter:.*",
    r"^Search Terms:.*",
    r"^Search Type:.*",
    r"^Content Type.*",
    r"^http[s]?://\\S+",
    r"^Page \\d+ of \\d+",
    r"^Load-Date:.*",
    r"^End of Document",
    r"^Classification$",
    r"^Language:.*",
    r"^Publication-Type:.*",
    r"^Subject:.*",
    r"^Industry:.*",
    r"^\\s*Bookmark_\\d+\\s*$"
]

HEADER_REGEXES = [re.compile(pat, flags=re.IGNORECASE) for pat in LEXIS_HEADER_PATTERNS]

def clean_lines(lines: List[str]) -> List[str]:
    out = []
    for ln in lines:
        s = ln.strip()
        if not s:
            out.append("")
            continue
        if any(rx.match(s) for rx in HEADER_REGEXES):
            continue
        s = re.sub(r"\\s+", " ", s)
        out.append(s)
    txt = "\\n".join(out)
    txt = re.sub(r"\\n{3,}", "\\n\\n", txt)
    return [ln for ln in txt.split("\\n")]
    
def pdf_to_text(path: str) -> str:
    texts = []
    with pdfplumber.open(path) as pdf:
        for page in pdf.pages:
            t = page.extract_text(x_tolerance=1, y_tolerance=1) or ""
            if t:
                texts.append(t)
    raw = "\\n".join(texts)
    raw = raw.replace("\\r\\n", "\\n").replace("\\r", "\\n")
    lines = raw.split("\\n")
    lines = clean_lines(lines)
    return "\\n".join(lines)

def normalize_text(s: str) -> str:
    if not s:
        return s
    # snelle mojibake-detectie en herstel (bv. 'patiÃ«nt' → 'patiënt')
    if re.search(r"[ÂÃ]|Ã.|â€|â€™|â€œ|â€�|â€“|â€”", s):
        try:
            s = s.encode("latin-1", "ignore").decode("utf-8", "ignore")
        except Exception:
            pass
    # unicode normalisatie + wat typografische tekens rechtzetten
    s = unicodedata.normalize("NFKC", s)
    s = (s.replace("“", "\"").replace("”", "\"")
           .replace("‘", "'").replace("’", "'")
           .replace("–", "-").replace("—", "-")
           .replace("\u00ad", ""))  # soft hyphen
    return re.sub(r"\s+", " ", s).strip()

In [4]:
STOP_MARKERS = [
    r"^Classification$", r"^End of Document$", r"^Load-Date:", r"^Subject:", r"^Industry:",
    r"^Language:", r"^Publication-Type:", r"^Graphic$", r"^Bookmark_\d+\s*$"
]
STOP_RX = re.compile("|".join(STOP_MARKERS), re.IGNORECASE)

def extract_title_lead_body(clean_text: str):
    """
    LexisNexis-structuur:
      [koppen/metadata] ... 
      Title (vaak 1-2x herhaald)
      Krant / datum / sectie / lengte / byline ...
      Body
      <artikeltekst (meerdere alinea's, soms paginabreaks)>
      Classification / End of Document / etc.

    Output:
      title: 1 regel
      lead:  korte samenvatting (eerste paragraaf na Body, overslaat 1-woord labels zoals 'Column', 'Geneesmiddelen')
      body:  rest tot aan STOP_MARKERS
    """
    # 1) naar regels
    lines = [ln.strip() for ln in clean_text.replace("\r", "\n").split("\n")]
    lines = [ln for ln in lines if ln is not None]  # behoud lege regels als scheiding

    # 2) vind titelkandidaat (eerste niet-lege regel die NIET 'Page x of y' / URL / About is)
    def is_noise_title(l):
        if not l: return True
        if re.match(r"^Page \d+ of \d+$", l): return True
        if re.match(r"^http[s]?://", l, re.I): return True
        if "About LexisNexis" in l or "Privacy Policy" in l or "Terms" in l: return True
        return False

    first_nonempty = next((i for i,l in enumerate(lines) if l and not is_noise_title(l)), None)
    title = lines[first_nonempty] if first_nonempty is not None else ""

    # 3) vind de EERSTE 'Body' regel (dit markeert begin van inhoud)
    body_idx = next((i for i,l in enumerate(lines) if l.strip().lower() == "body"), None)
    if body_idx is None:
        # fallback: soms staat 'Body' met extra tekst eronder; probeer een zachtere match
        body_idx = next((i for i,l in enumerate(lines) if re.fullmatch(r"\s*Body\s*", l, re.I)), None)

    # als geen Body gevonden: alles na title als body (zeldzaam)
    start_idx = (body_idx + 1) if body_idx is not None else ((first_nonempty + 1) if first_nonempty is not None else 0)

    # 4) knip af bij eerste STOP_MARKER
    end_idx = None
    for j in range(start_idx, len(lines)):
        if STOP_RX.match(lines[j] or ""):
            end_idx = j
            break
    content_lines = lines[start_idx:end_idx] if end_idx else lines[start_idx:]

    # 5) verwijder bekende tussenkopjes / 1-woord labels direct na Body (bv. 'Column', 'Geneesmiddelen')
    while content_lines and re.fullmatch(r"[A-Za-zÀ-ÿ\-’'`]+", content_lines[0]):
        # laat staan als het duidelijk een zin is (eindigt op .?!)
        if re.search(r"[.!?]$", content_lines[0]): break
        # anders overslaan (één-woord of korte rubriek)
        content_lines.pop(0)

    # 6) maak paragrafen (lege regel = scheiding; als die ontbreken, per “lege regel” simuleren op dubbele spaties)
    raw = "\n".join(content_lines)
    # normaliseer meerdere lege regels
    raw = re.sub(r"\n{3,}", "\n\n", raw).strip()
    paras = [p.strip() for p in re.split(r"\n\s*\n", raw) if p.strip()]
    if not paras:
        # fallback: forceer paragrafen grofweg op zinsafsluiting
        paras = [p.strip() for p in re.split(r"(?<=[.!?])\s+", raw) if p.strip()]

    # 7) lead = eerste paragraaf (max 2-3 zinnen), body = rest
    if paras:
        # knip lead op 2-3 zinnen
        sents = re.split(r"(?<=[.!?])\s+", paras[0])
        lead = " ".join(sents[:3]).strip()
        remainder = " ".join(sents[3:]).strip()
        body_paras = ([remainder] if remainder else []) + paras[1:]
        body = "\n\n".join(body_paras).strip()
    else:
        lead, body = "", ""

    title = normalize_text(title)
    lead  = normalize_text(lead)
    body  = normalize_text(body)

    return title.strip(), lead, body

# Test PDF extraction

In [5]:
test_files = [
    "data/Bericht 007 Nederlandse pati_nt wacht te lang op betere medicijnen tegen kanker.pdf",
    "data/Bericht 010_Nieuwe kankermedicijnen leveren meer financi_le winst op dan gezondheidswinst.pdf",
    "data/Bericht 011_Hoe controleer je verstopte moedervlekken_.pdf",
    "data/Bericht 016_Ik vind het erg als _n infuus van 25.000 euro wordt weggegooid_.pdf",
]

for fp in test_files:
    txt = pdf_to_text(fp)           # jouw bestaande pdf->text + clean
    title, lead, body = extract_title_lead_body(txt)
    print("\n===", os.path.basename(fp), "===")
    print("TITLE:", title[:120])
    print("LEAD :", lead)
    print("BODY len:", len(body), "chars")
    print("BODY preview:", body[:300].replace("\n"," ") + " ...")


=== Bericht 007 Nederlandse pati_nt wacht te lang op betere medicijnen tegen kanker.pdf ===
TITLE: Nederlandse patiënt wacht te lang op betere medicijnen tegen kanker
LEAD : Wat een prachtig bericht onlangs, dat meer kankerpatiënten de afgelopen decennia bleven leven. Twee derde van de patiënten met de diagnose kanker leeft na vijf jaar nog. Een vooruitgang die volgens het Integraal Kankercentrum Nederland mede te danken is aan innovatieve geneesmiddelen tegen gevorderde en uitgezaaide kanker.
BODY len: 3697 chars
BODY preview: Nog niet voor alle soorten, maar in ieder geval voor huid-, long-, prostaat-, bloed-, nier- en blaaskanker. Dat zijn in aantal niet de minste. Maar het is jammer dat het zo lang duurt voordat dergelijke geneesmiddelen na goedkeuring door de Amerikaanse autoriteiten in de Nederlandse praktijk terecht ...

=== Bericht 010_Nieuwe kankermedicijnen leveren meer financi_le winst op dan gezondheidswinst.pdf ===
TITLE: Nieuwe kankermedicijnen leveren meer financiële wi

In [6]:
def make_chunks_by_tokens(text: str, tokenizer, max_tokens: int = 400, stride: int = 50) -> List[str]:
    if not text.strip():
        return []
    toks = tokenizer.encode(text, add_special_tokens=False)
    chunks = []
    i = 0
    while i < len(toks):
        window = toks[i:i+max_tokens]
        if not window:
            break
        chunk = tokenizer.decode(window, skip_special_tokens=True)
        chunks.append(chunk)
        if i + max_tokens >= len(toks):
            break
        i += max_tokens - stride
    return chunks

def chunk_body(text: str, tokenizer, prefer_paragraphs: bool = True, max_tokens: int = 400, stride: int = 50) -> List[str]:
    if not text.strip():
        return []
    if prefer_paragraphs:
        paras = re.split(r"\\n\\s*\\n", text.strip())
        paras = [p.strip() for p in paras if p.strip()]
        chunks = []
        for p in paras:
            if len(tokenizer.encode(p, add_special_tokens=False)) <= max_tokens:
                chunks.append(p)
            else:
                chunks.extend(make_chunks_by_tokens(p, tokenizer, max_tokens=max_tokens, stride=stride))
        return chunks
    else:
        return make_chunks_by_tokens(text, tokenizer, max_tokens=max_tokens, stride=stride)


In [7]:
# === Alleen de RobBERT v2 sentiment pipeline laden ===
ROB_NAME = "DTAI-KULeuven/robbert-v2-dutch-sentiment"

def load_pipe(name, tries=2):
    last = None
    for i in range(tries):
        try:
            clf = pipeline(
                task="sentiment-analysis",
                model=name,
                tokenizer=name,
                top_k=None,          # return_all_scores vervangen
                truncation=True
            )
            return clf
        except Exception as e:
            last = e
    raise last

rob_pipe = load_pipe(ROB_NAME)
TOKENIZER = AutoTokenizer.from_pretrained(ROB_NAME)

Device set to use cpu


In [8]:
LABEL_MAP = {
    "POSITIVE": "positief", "NEGATIVE": "negatief", "NEUTRAL": "neutraal",
    "Positive": "positief", "Negative": "negatief", "Neutral": "neutraal",
    "positief": "positief", "negatief": "negatief", "neutraal": "neutraal"
}

def normalize_pnn(probs: dict) -> dict:
    import numpy as np
    arr = np.array([
        probs.get("positief", 0.0),
        probs.get("negatief", 0.0),
        probs.get("neutraal", 0.0)
    ], dtype=float)
    s = arr.sum()
    if s <= 0:
        return {"positief":0.0, "negatief":0.0, "neutraal":1.0}
    arr = arr / s
    return {"positief": float(arr[0]), "negatief": float(arr[1]), "neutraal": float(arr[2])}

def score_text_with_pipe(text: str, clf) -> dict:
    text = (text or "").strip()
    if not text:
        return {"positief":0.0, "negatief":0.0, "neutraal":1.0}
    out = clf(text, truncation=True)
    scores = out[0]  # top_k=None -> lijst van dicts
    probs = {"positief":0.0, "negatief":0.0, "neutraal":0.0}
    for item in scores:
        lab = LABEL_MAP.get(item["label"])
        if lab:
            probs[lab] = float(item["score"])
    return normalize_pnn(probs)

MAX_MODEL_TOKENS = 512
HEADROOM = 8
SAFE_MAX = MAX_MODEL_TOKENS - HEADROOM

def token_chunks(text, tokenizer, max_tokens=SAFE_MAX, stride=50):
    if not text or not text.strip():
        return []
    ids = tokenizer.encode(text, add_special_tokens=False)
    if len(ids) <= max_tokens:
        return [text]
    out, i = [], 0
    while i < len(ids):
        j = min(i + max_tokens, len(ids))
        chunk_ids = ids[i:j]
        out.append(tokenizer.decode(chunk_ids, clean_up_tokenization_spaces=True))
        if j >= len(ids): break
        i = max(j - stride, i + 1)
    return out

def aggregate_article_with_pipe_binary(title, lead, body_chunks, clf, tokenizer,
                                       weights=None):
    """
    Alleen POS/NEG. Negeert neutraal volledig.
    - Titel/lead/body worden gechunked zoals voorheen.
    - Weeg t/l/b via weights (default title=2.0, lead=1.0, body=1.0).
    - Label = 'positief' als p_pos >= p_neg, anders 'negatief'.
    """
    if weights is None:
        from dataclasses import dataclass
        @dataclass
        class W: title: float = 2.0; lead: float = 1.0; body: float = 1.0
        weights = W()

    parts = []

    # Titel
    t_chunks = token_chunks(title or "", tokenizer, max_tokens=SAFE_MAX, stride=50)
    w_t_each = (weights.title / max(len(t_chunks), 1)) if t_chunks else 0.0
    for t in t_chunks:
        if t.strip():
            parts.append((t.strip(), w_t_each))

    # Lead
    l_chunks = token_chunks(lead or "", tokenizer, max_tokens=SAFE_MAX, stride=50)
    w_l_each = (weights.lead / max(len(l_chunks), 1)) if l_chunks else 0.0
    for l in l_chunks:
        if l.strip():
            parts.append((l.strip(), w_l_each))

    # Body (al gechunked upstream)
    if body_chunks:
        w_b_each = weights.body / len(body_chunks)
        for ch in body_chunks:
            ch = (ch or "").strip()
            if ch:
                parts.append((ch, w_b_each))

    if not parts:
        # lege tekst -> kies 'negatief' conservatief of geef pos=neg=0.5
        return {"p_pos":0.5, "p_neg":0.5, "label":"negatief"}

    acc_pos = 0.0
    acc_neg = 0.0
    total_w = 0.0

    for txt, w in parts:
        out = clf(txt, truncation=True, max_length=MAX_MODEL_TOKENS, padding=False)
        scores = out[0]  # lijst met dicts
        p_pos = p_neg = 0.0
        for item in scores:
            lab = LABEL_MAP.get(item["label"])
            if lab == "positief":
                p_pos = float(item["score"])
            elif lab == "negatief":
                p_neg = float(item["score"])
            # 'neutraal' negeren we bewust

        # Her-normaliseer over POS/NEG alleen (optioneel, maar aan te raden)
        s = p_pos + p_neg
        if s > 0:
            p_pos2 = p_pos / s
            p_neg2 = p_neg / s
        else:
            # als model iets geks geeft, maak gelijk verdeeld
            p_pos2 = p_neg2 = 0.5

        acc_pos += p_pos2 * w
        acc_neg += p_neg2 * w
        total_w += w

    if total_w <= 0:
        total_w = 1.0
    p_pos_bin = acc_pos / total_w
    p_neg_bin = acc_neg / total_w

    label = "positief" if p_pos_bin >= p_neg_bin else "negatief"
    return {"p_pos": float(p_pos_bin), "p_neg": float(p_neg_bin), "label": label}

# Analysis

In [27]:
rows = []
pdf_files = [f for f in sorted(os.listdir(DATA_DIR)) if f.lower().endswith('.pdf')]
if not pdf_files:
    print(f"[INFO] Geen PDF-bestanden gevonden in '{DATA_DIR}'.")
else:
    for fname in tqdm(pdf_files, desc="PDFs verwerken"):
        fpath = os.path.join(DATA_DIR, fname)
        clean_txt = pdf_to_text(fpath)
        title, lead, body = extract_title_lead_body(clean_txt)

        body_chunks = chunk_body(
            body,
            TOKENIZER,
            prefer_paragraphs=True,
            max_tokens=MAX_TOKENS,
            stride=STRIDE
        )

        rob_bin = aggregate_article_with_pipe_binary(
            title, lead, body_chunks,
            rob_pipe, TOKENIZER
        )

        rows.append({
            "id": extract_id_from_filename(fname) if 'extract_id_from_filename' in globals() else fname,
            "file": fname,
            "title": title,
            "lead": lead,

            # Alleen binaire output
            "rob_p_pos": rob_bin["p_pos"],
            "rob_p_neg": rob_bin["p_neg"],
            "rob_label": rob_bin["label"],  # nu altijd 'positief' of 'negatief'
        })

df = pd.DataFrame(rows)
print(df.head())

PDFs verwerken: 100%|██████████████████████████████████████████████████████████████████| 70/70 [02:36<00:00,  2.23s/it]

                                                  id  \
0  Bericht 007 Nederlandse pati_nt wacht te lang ...   
1  Bericht 010_Nieuwe kankermedicijnen leveren me...   
2  Bericht 011_Hoe controleer je verstopte moeder...   
3  Bericht 016_Ik vind het erg als _n infuus van ...   
4  Bericht 021_Wachtlijsten en personeelstekort_ ...   

                                                file  \
0  Bericht 007 Nederlandse pati_nt wacht te lang ...   
1  Bericht 010_Nieuwe kankermedicijnen leveren me...   
2  Bericht 011_Hoe controleer je verstopte moeder...   
3  Bericht 016_Ik vind het erg als _n infuus van ...   
4  Bericht 021_Wachtlijsten en personeelstekort_ ...   

                                               title  \
0  Nederlandse patiënt wacht te lang op betere me...   
1  Nieuwe kankermedicijnen leveren meer financiël...   
2         Hoe controleer je verstopte moedervlekken?   
3  'Ik vind het erg als 'n infuus van 25.000 euro...   
4  Wachtlijsten en personeelstekort: het 'zorg

In [28]:
df.to_csv("sentiment_results.csv", index=False, encoding="utf-8")
print("[DONE] Geschreven naar sentiment_results.csv")

[DONE] Geschreven naar sentiment_results.csv


# Classification Results

In [9]:
# === Load results (RobBERT only) ===
import re
import pandas as pd

df = pd.read_csv("sentiment_results.csv")

# ---------------------------
# 1) Parse ID (robust)
#    - Werkt voor 'Bericht 007_*', 'Bericht_007 ...', of als id al numeriek is
# ---------------------------
def extract_id(val):
    # Als al numeriek:
    try:
        return int(val)
    except Exception:
        pass
    s = str(val)
    m = re.search(r'bericht[_\s-]*(\d+)', s, flags=re.IGNORECASE)
    if m:
        return int(m.group(1))
    # fallback: neem eerste getal dat voorkomt
    m2 = re.search(r'(\d+)', s)
    return int(m2.group(1)) if m2 else None

if 'id' in df.columns:
    df['id'] = df['id'].apply(extract_id)
else:
    # als er geen 'id' kolom is, maak er één op basis van 'file' of index
    if 'file' in df.columns:
        df['id'] = df['file'].apply(extract_id)
    else:
        df['id'] = range(1, len(df) + 1)

# ---------------------------
# 2) Alleen relevante kolommen (RobBERT)
# ---------------------------
if 'rob_label' not in df.columns:
    raise KeyError("Column 'rob_label' not found in sentiment_results.csv")

# labels netjes lowercased (verwacht: positief/negatief/neutraal)
df['rob_label'] = df['rob_label'].astype(str).str.lower()

df_labels = df[['id', 'rob_label']].copy()

# ---------------------------
# 3) One-hot encoding voor RobBERT
#    >>> levert kolommen: robbert_positief, robbert_negatief, robbert_neutraal
# ---------------------------
df_encoded = pd.get_dummies(df_labels, columns=['rob_label'], prefix='robbert')

# ---------------------------
# 4) Bekijken
# ---------------------------
print(df_encoded.head())

   id  robbert_negatief  robbert_positief
0   7              True             False
1  10              True             False
2  11             False              True
3  16              True             False
4  21             False              True


In [10]:
print(df_encoded.tail())

     id  robbert_negatief  robbert_positief
65  287             False              True
66  296             False              True
67  300              True             False
68  306             False              True
69   55             False              True


In [11]:
summary = {}

for model_prefix in ['robbert']:
    summary[model_prefix] = {
        'positief': df_encoded.filter(like=f"{model_prefix}_positief").sum().values[0],
        #'neutraal': df_encoded.filter(like=f"{model_prefix}_neutraal").sum().values[0],
        'negatief': df_encoded.filter(like=f"{model_prefix}_negatief").sum().values[0]
    }

# Zet om naar DataFrame
summary_df = pd.DataFrame(summary).T
summary_df.index.name = "Model"

# Resultaat bekijken
print(summary_df)

         positief  negatief
Model                      
robbert        52        18


# Analyse tov human sentiment (maaike)

In [12]:
# === Evaluatie: Human vs RobBERT (binaire evaluatie: positief vs negatief) ===
import re
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix

# 1) Data inladen
df_h = pd.read_excel("Human_Sentiment.xlsx")   # kolommen: Artikel, Sentiment
df_n = pd.read_csv("sentiment_results.csv")    # bevat 'id' + rob_label (pos/neg)

# 2) ID-parsing
def extract_id(s):
    s = str(s)
    m = re.search(r'bericht[_\s-]*(\d+)', s, flags=re.IGNORECASE)
    if m: return int(m.group(1))
    m2 = re.search(r'(\d+)', s)
    return int(m2.group(1)) if m2 else None

if 'id' in df_n.columns:
    df_n['id'] = df_n['id'].apply(extract_id)
elif 'file' in df_n.columns:
    df_n['id'] = df_n['file'].apply(extract_id)
else:
    df_n['id'] = range(1, len(df_n) + 1)

if not pd.api.types.is_integer_dtype(df_h['Artikel']):
    try:
        df_h['Artikel'] = df_h['Artikel'].astype(int)
    except Exception:
        df_h['Artikel'] = df_h['Artikel'].apply(extract_id)

# 3) Human labels normaliseren en filteren (alleen pos/neg)
df_h['Human_Label'] = (
    df_h['Sentiment']
      .astype(str).str.strip().str.lower()
)
# Drop 'mixed' en 'neutral' (vallen af)
df_h = df_h[df_h['Human_Label'].isin({'positief','negatief'})].copy()

# 4) Merge
dfm = pd.merge(df_h, df_n, left_on='Artikel', right_on='id', how='inner')

# 5) RobBERT kolom
if 'rob_label' not in dfm.columns:
    raise KeyError("rob_label niet gevonden in sentiment_results.csv (verwacht binaire output).")
y_true = dfm['Human_Label'].astype(str).str.lower()
y_pred = dfm['rob_label'].astype(str).str.lower()

# 6) Evaluatie (binaire labels)
labels_order = ['positief','negatief']
print("\n=== RobBERT (binaire evaluatie) ===")
print(classification_report(
    y_true, y_pred,
    labels=labels_order,
    target_names=labels_order,
    digits=3
))

cm = confusion_matrix(y_true, y_pred, labels=labels_order)
cm_df = pd.DataFrame(
    cm,
    index=[f"True {l}" for l in labels_order],
    columns=[f"Pred {l}" for l in labels_order]
)
print("Confusion matrix:")
print(cm_df)



=== RobBERT (binaire evaluatie) ===
              precision    recall  f1-score   support

    positief      0.745     0.875     0.805        40
    negatief      0.722     0.520     0.605        25

    accuracy                          0.738        65
   macro avg      0.733     0.698     0.705        65
weighted avg      0.736     0.738     0.728        65

Confusion matrix:
               Pred positief  Pred negatief
True positief             35              5
True negatief             12             13


In [13]:
errors = dfm[(dfm['Human_Label'] != dfm['rob_label'])]
print(errors[['id', 'title', 'rob_label', 'Human_Label']].to_string(index=False))

 id                                                                                                      title rob_label Human_Label
  7                                        Nederlandse patiënt wacht te lang op betere medicijnen tegen kanker  negatief    positief
 16                                           'Ik vind het erg als 'n infuus van 25.000 euro wordt weggegooid'  negatief    positief
 26                          Tijd om te kiezen: dure behandelingen of voldoende zorg voor ouderen ; Commentaar  positief    negatief
 63                                                                          Langer lijden of waardig sterven?  positief    negatief
 66                                                               'Mijn belangrijkste vraag: wat wil je echt?'  positief    negatief
 69                                                       Geef nieuwe medicijnen geen groen licht, maar oranje  positief    negatief
 74                                         Goed dat grens wordt gest